In [ ]:
import os
import time

import numpy as np

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt

import skimage
import skimage.io as sio

import fracatal
from fracatal.functional import ft_convolve

import IPython

In [ ]:
"""
functions for building CA physics
"""

def make_gaussian(a, m, s):
    eps = 1e-9
    def gaussian(x):
        
        return a * np.exp(-((x-m)/(eps+s))**2 / 2)

    return gaussian
    
def make_mixed_gaussian(amplitudes, means, std_devs):
    
    def gaussian_mixture(x):
        results = 0.0 * x
        eps = 1e-9 # prevent division by 0
        
        for a, m, s in zip(amplitudes, means, std_devs):
            my_gaussian = make_gaussian(a, m, s)
            results += my_gaussian(x)
        
        return results

    return gaussian_mixture

def make_kernel_field(kernel_radius, dim=126):

    #dim = kernel_radius * 2 + 1
    

    x =  np.arange(-dim / 2, dim / 2 + 1, 1)
    xx, yy = np.meshgrid(x,x)

    rr = np.sqrt(xx**2 + yy**2) / kernel_radius

    return rr

def make_update_function(mean, standard_deviation):

    my_gaussian = make_gaussian(1.0, mean, standard_deviation)
    
    def lenia_update(x):
        """
        lenia update
        """
        return 2 * my_gaussian(x) - 1

    return lenia_update

def make_update_step(update_function, kernel, dt, clipping_function = lambda x: x, decimals=None):

    if decimals is not None:
        r = lambda x: np.round(x, decimals=decimals)
    else:
        r = lambda x: x
        
    def update_step(grid):

        
        neighborhoods = r(ft_convolve(r(grid), r(kernel)))
        dgrid_dt = r(update_function(neighborhoods))

        new_grid = r(clipping_function(r(grid) + dt * dgrid_dt))

        return new_grid

    return update_step

def make_make_kernel_function(amplitudes, means, standard_deviations, dim=126):
    
    def make_kernel(kernel_radius):
                
        gm = make_mixed_gaussian(amplitudes, means, standard_deviations)
        rr = make_kernel_field(kernel_radius, dim=dim)
        kernel = gm(rr)[None,None,:,:]
        kernel /= kernel.sum()

        return kernel

    return make_kernel

# smooth life
def sigmoid_1(x, mu, alpha, gamma=1):
    return 1 / (1 + np.exp(-4 * (x - mu) / alpha))

def get_smooth_steps_fn(intervals, alpha=0.0125):
    """
    construct an update function from intervals.
    input intervals is a list of lists of interval bounds,
    each element contains the start and end of an interval. 

    # this code was adopated from rivesunder/yuca
    """
    
    def smooth_steps_fn(x):
        
        result = np.zeros_like(x)
        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1 - sigmoid_1(x, bounds[1], alpha))
        
        return result
    
    return smooth_steps_fn

def make_make_smoothlife_kernel_function(r_inner, r_outer):
    """
    r_inner and r_outer are 1/3. and 1.05 for Rafler's SmoothLife glider configuration
    """
    
    def make_smoothlife_kernel(kernel_radius=10):
        
        rr = make_kernel_field(kernel_radius)
        kernel = gm(rr)[None,None,:,:]
        kernel /= kernel.sum()

        return kernel
    
    return make_smoothlife_kernel
    
    def forward(self, x):

        kernel = np.zeros_like(x)

        kernel[x >= self.r_i] = 1.0
        kernel[x > self.r_a] = 0.0

        return torch.tensor(kernel)

def make_smooth_interval(alpha=0.1470, intervals=[[0.2780, 0.3650]]):

    def smooth_interval(x):

        result = 0.0 * x

        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1-sigmoid_1(x, bounds[1], alpha))

        return 2 * result - 1

    return smooth_interval
                                                        
# smooth life
def make_make_smoothlife_kernel_function(r_inner, r_outer, dim=126):
    """
    r_inner and r_outer are 1/3. and 1.05 for Rafler's SmoothLife glider configuration
    """
    
    def make_smoothlife_kernel(kernel_radius=10):
        
        rr = make_kernel_field(kernel_radius, dim=dim)
        kernel = np.ones_like(rr)
        kernel[rr < r_inner] = 0.0
        kernel[rr >= r_outer] = 0.0
        
        kernel = (kernel / kernel.sum())[None,None,:,:]

        return kernel
    
    return make_smoothlife_kernel

def make_smooth_interval(alpha=0.1470, intervals=[[0.2780, 0.3650]]):

    def smooth_interval(x):

        result = 0.0 * x

        for bounds in intervals:
            result += sigmoid_1(x, bounds[0], alpha) * (1-sigmoid_1(x, bounds[1], alpha))

        return 2 * result - 1

    return smooth_interval
    
def make_smoothlife_update_function(intervals=[[0.2780, 0.3650]], \
                                    alpha=0.028):

    smooth = make_smooth_interval(alpha=alpha, intervals=intervals)
    
    def smoothlife_update(x):
        """
        smoothlife half-update
        """
        return 2 * smooth(x) - 1

    return smoothlife_update

def make_smoothlife_update_step(genesis_function, persistence_function, \
                                kernel, inner_kernel, dt, clipping_function = lambda x: x, \
                                decimals=None):

    if decimals is not None:
        r = lambda x: np.round(x, decimals=decimals)
    else:
        r = lambda x: x
        
    def update_step(grid):

        neighborhoods = r(ft_convolve(r(grid), r(kernel)))
        inner = r(ft_convolve(r(grid), r(inner_kernel)))

        genesis = r(genesis_function(neighborhoods))
        persistence = r(persistence_function(neighborhoods))
        dgrid_dt = (1-inner) * genesis + inner * persistence
                 
        new_grid = r(clipping_function(r(grid) + dt * dgrid_dt))

        return new_grid

    return update_step


In [ ]:
"""
animation functions
"""

def get_fig(grid):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1)
    
    subplot_0 = ax.imshow(grid.squeeze(), cmap="magma")
    
    return fig, ax

def update_frame(ii):
    
    global grid
    
    subplot_0.set_array(grid.squeeze())
    
    grid = update_step(grid)


In [ ]:
def stability_sweep(dts, krs, starting_grid, pattern, make_kernel, my_update, max_t, \
                    persistence_update=None, \
                    make_inner_kernel=None, \
                    max_steps=1000, \
                    max_growth=2, \
                    min_growth=0.5, \
                    k0=31):


    clipping_fn = lambda x: np.clip(x, 0.0, 1.0)
    results = np.zeros((dts.shape[0], krs.shape[0],3))
    max_growth = 1.5
    min_growth = 0.5    


    native_dim_h = pattern.shape[-2]
    native_dim_w = pattern.shape[-1]
    
    for ii, dt in enumerate(dts):
        for jj, kr in enumerate(krs):
            
            kernel = make_kernel(kr)
            
            if make_inner_kernel is not None:
                inner_kernel = make_inner_kernel(k0)
                update_step = make_smoothlife_update_step(my_update, persistence_update, \
                        kernel, inner_kernel, dt, \
                        clipping_fn)
            else:
                update_step = make_update_step(my_update, kernel, dt, clipping_fn)

            accumulated_t = 0.0
            total_steps = 0
            scale_factor = kr / k0

            dim_h = int(native_dim_h * scale_factor)
            dim_w = int(native_dim_w * scale_factor)
                    
            scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w))
            
            grid = starting_grid * 0.0
            grid[:,:,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]] = scaled_pattern

            starting_sum = grid.sum()
            
            explode = False
            vanish = False

            if(0):
                
                plt.figure()
                plt.imshow(scaled_pattern.squeeze())
                plt.show()
                
                plt.figure()
                plt.imshow(grid.squeeze())
                plt.show()
            
            while accumulated_t < max_t and total_steps <= max_steps:
 
                grid = update_step(grid)
                
                g = grid.sum() / starting_sum
                
                accumulated_t += dt
                total_steps += 1
                if g > max_growth:
                    explode = True
                    break
                if g < min_growth:
                    vanish = True
                    break
            if(0):
                plt.figure()
                plt.imshow(grid.squeeze())
                plt.show()
                
            if explode == True:
                results[ii,jj,0] = 1-accumulated_t / max_t
            elif vanish == True:
                results[ii,jj,2] = 1-accumulated_t / max_t
            else:
                results[ii,jj,1] = accumulated_t / max_t

    return results

    

# SmoothLife glider

In [ ]:

pattern_filepath = os.path.join("..", "patterns", "smoothlife_single.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")
plt.title("pattern")
plt.show()

In [ ]:
num_frames = 100
kr = 10
k0 = 10
dt= 0.90

kernel = make_sl_kernel(kr)
inner_kernel = make_sl_inner_kernel(k0)
update_step = make_smoothlife_update_step(gen, per, \
        kernel, inner_kernel, dt, \
        clipping_function = lambda x: np.clip(x, 0, 1.0), \
        decimals=None)


scale_factor = kr / k0
native_dim_h = pattern.shape[-2]
native_dim_w = pattern.shape[-1]
dim_h = int(native_dim_h * scale_factor)
dim_w = int(native_dim_w * scale_factor)
        
scaled_pattern = skimage.transform.resize(pattern, (1,1, dim_h, dim_w),order=5)

grid = np.zeros((1, 1, grid_dim, grid_dim))
grid[:,:,:scaled_pattern.shape[-2], :scaled_pattern.shape[-1]] = scaled_pattern

fig, ax = get_fig(grid[0])
plt.close()

#update_step = lambda x: cca(torch.tensor(x).float()).numpy()
IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
min_dt = 0.01
max_dt = 1.0
number_dt_steps = 32
min_kr = 6
max_kr = 63
number_kr_steps = 32

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 200

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_outer_kernel = make_make_smoothlife_kernel_function(r_inner, r_outer, dim = 126)
make_inner_kernel = make_make_smoothlife_kernel_function(0.0, r_inner, dim = 126)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_outer_kernel, gen, max_t, per, make_inner_kernel, k0=10, max_steps=1000)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

# _H. natans_

In [ ]:
# common setup for H. natans

# the neighborhood kernel
amplitudes = [0.5, 1.0, 0.6667]
means = [0.0938, 0.2814, 0.4690]
standard_deviations = [0.0330, 0.0330, 0.0330]
kernel_radius = 31

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
kernel = make_kernel(kernel_radius)

# the growth function
mean_g = 0.26
standard_deviation_g = 0.036

clipping_fn = lambda x: np.clip(x,0,1.0)
my_update = make_update_function(mean_g, standard_deviation_g)


# The Platonic Pattern: _Hydrogeminium natans_ pickle

In [ ]:
pattern_filepath = os.path.join("..", "patterns", "hydrogeminium_natans_pickle.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


In [ ]:
number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

dts = 0.1

clipping_fn = lambda x: np.clip(x,0,1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn, decimals=32)

num_frames = 100

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=10).to_jshtml())

In [ ]:
min_dt = 0.001
max_dt = 1.0
number_dt_steps = 64
min_kr = 5
max_kr = 63
number_kr_steps = 64

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

# The Non-Platonic _H. natans_ wobbler

In [ ]:
#
pattern_filepath = os.path.join("..", "patterns", "hydrogeminium_natans_wobbler.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


In [ ]:
number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

dts = 0.38

clipping_fn = lambda x: np.clip(x,0,1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn, decimals=32)

num_frames = 100

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=100).to_jshtml())

In [ ]:
min_dt = 0.001
max_dt = 1.0
number_dt_steps = 64
min_kr = 5
max_kr = 63
number_kr_steps = 64

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.43
max_dt = 0.46
number_dt_steps = 64
min_kr = 43
max_kr = 46
number_kr_steps = 64

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations, dim=190)
t0 = time.time()

results = stability_sweep(dts, krs, grid, pattern, make_kernel, my_update, max_t, max_steps=100)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)